## Test

*A notebook by Sofia wherein some simple, test data is subjected to the same
analysis pipeline as we are applying to real observational data, in order to 
confirm that the routines work as expected.*

In [1]:
import os
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
from ccdproc import ImageFileCollection, Combiner, combine, subtract_dark, flat_correct

In [2]:
light_exposure = 30 * u.second
dark_exposure = 30 * u.second
flat_exposure = 1 * u.second
bias_exposure = 1 * u.second

In [3]:
light1 = CCDData(np.array([[3, 2],[4, 5]]), unit=u.adu)
light2 = CCDData(np.array([[5, 7],[4, 3]]), unit=u.adu)
lights = [light1, light2]

In [4]:
dark1 = CCDData(np.array([[1, 1],[1, 2]]), unit=u.adu)
dark2 = CCDData(np.array([[2, 0],[1, 2]]), unit=u.adu)
dark3 = CCDData(np.array([[0, 0],[0, 1]]), unit=u.adu)

dark_average = combine([dark1, dark2, dark3])
dark_median = combine([dark1, dark2, dark3], method='median')

In [5]:
flat1 = CCDData(np.array([[2, 2],[4, 3]]), unit=u.adu)
flat2 = CCDData(np.array([[6, 5],[5, 6]]), unit=u.adu)
flat3 = CCDData(np.array([[6, 4],[3, 2]]), unit=u.adu)

flat_average = combine([flat1, flat2, flat3])
flat_median = combine([flat1, flat2, flat3], method='median')

In [6]:
bias1 = CCDData(np.array([[1, 1],[3, 2]]), unit=u.adu)
bias2 = CCDData(np.array([[3, 2],[1, 3]]), unit=u.adu)
bias3 = CCDData(np.array([[3, 2],[0, 1]]), unit=u.adu)

bias_average = combine([bias1, bias2, bias3])
bias_median = combine([bias1, bias2, bias3], method='median')

In [7]:
dark_median

CCDData([[1., 0.],
         [1., 2.]], unit='adu')

In [8]:
lights_subtracted_average = [subtract_dark(light, dark_average, data_exposure=light_exposure, dark_exposure=dark_exposure, scale=False) for light in lights]
lights_subtracted_median = [subtract_dark(light, dark_median, data_exposure=light_exposure, dark_exposure=dark_exposure, scale=False) for light in lights]

In [9]:
flat_subtracted_average = subtract_dark(flat_average, bias_average, data_exposure=flat_exposure, dark_exposure=bias_exposure, scale=False)
flat_subtracted_median = subtract_dark(flat_median, bias_median, data_exposure=flat_exposure, dark_exposure=bias_exposure, scale=False)

In [10]:
lights_calibrated_average = [flat_correct(l, flat_subtracted_average) for l in lights_subtracted_average]
lights_calibrated_median = [flat_correct(l, flat_subtracted_median) for l in lights_subtracted_average]

In [11]:
lights_calibrated_average

[CCDData([[1.85714286, 1.80555556],
          [2.70833333, 4.33333333]], unit='adu'),
 CCDData([[3.71428571, 7.22222222],
          [2.70833333, 1.73333333]], unit='adu')]

In [12]:
expected_calibrated_average = CCDData(np.array([[1.8889, 2.6111],[1.6944, 2.7778]]), unit=u.adu)

In [13]:
lights_calibrated_median

[CCDData([[1.5  , 1.875],
          [2.5  , 7.5  ]], unit='adu'),
 CCDData([[3. , 7.5],
          [2.5, 3. ]], unit='adu')]

In [14]:
expected_calibrated_median = CCDData(np.array([[2, 1],[3, 3]]), unit=u.adu)

In [15]:
dark_average

CCDData([[1.        , 0.33333333],
         [0.66666667, 1.66666667]], unit='adu')